In [162]:
# pip install spacy
# python -m spacy download de_core_news_lg
import pandas as pd
import spacy
sp = spacy.load('de_core_news_lg')

In [163]:
skills = pd.read_csv('./data/competencies/skills_de.csv')
skills.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13891 entries, 0 to 13890
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   conceptType     13891 non-null  object 
 1   conceptUri      13891 non-null  object 
 2   skillType       13890 non-null  object 
 3   reuseLevel      13891 non-null  object 
 4   preferredLabel  13891 non-null  object 
 5   altLabels       2928 non-null   object 
 6   hiddenLabels    32 non-null     object 
 7   status          13891 non-null  object 
 8   modifiedDate    13891 non-null  object 
 9   scopeNote       0 non-null      float64
 10  definition      0 non-null      float64
 11  inScheme        13891 non-null  object 
 12  description     13891 non-null  object 
dtypes: float64(2), object(11)
memory usage: 1.4+ MB


In [164]:
skills

,conceptType,conceptUri,skillType,reuseLevel,preferredLabel,altLabels,hiddenLabels,status,modifiedDate,scopeNote,definition,inScheme,description
0,KnowledgeSkillCompetence,http://data.europa.eu/esco/skill/0005c151-5b5a...,skill/competence,sector-specific,Musikpersonal verwalten,NaN,NaN,released,2016-12-20T17:43:43Z,NaN,NaN,http://data.europa.eu/esco/concept-scheme/skil...,Zuweisen und Verwalten der Aufgaben des Person...
1,KnowledgeSkillCompetence,http://data.europa.eu/esco/skill/00064735-8fad...,skill/competence,occupation-specific,Strafvollzugsverfahren beaufsichtigen,NaN,NaN,released,2016-12-20T20:17:49Z,NaN,NaN,http://data.europa.eu/esco/concept-scheme/memb...,Überwachen des Betriebs einer Justizvollzugsan...
2,KnowledgeSkillCompetence,http://data.europa.eu/esco/skill/000709ed-2be5...,skill/competence,sector-specific,nicht unterdrückende Praktiken anwenden,NaN,NaN,released,2016-12-20T19:18:19Z,NaN,NaN,http://data.europa.eu/esco/concept-scheme/skil...,"Ermitteln von Repressionen in Gesellschaften, ..."
3,KnowledgeSkillCompetence,http://data.europa.eu/esco/skill/0007bdc2-dd15...,skill/competence,sector-specific,Einhaltung von Vorschriften von Eisenbahnfahrz...,NaN,NaN,released,2016-12-20T20:02:19Z,NaN,NaN,http://data.europa.eu/esco/concept-scheme/skil...,"Kontrollieren von Fahrzeugen, Komponenten und ..."
4,KnowledgeSkillCompetence,http://data.europa.eu/esco/skill/00090cc1-1f27...,skill/competence,cross-sector,verfügbare Dienste ermitteln,NaN,NaN,released,2016-12-20T20:15:17Z,NaN,NaN,http://data.europa.eu/esco/concept-scheme/memb...,Ermitteln der verschiedenen verfügbaren Dienst...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13886,KnowledgeSkillCompetence,http://data.europa.eu/esco/skill/ffef5eb3-a15e...,skill/competence,sector-specific,berufliche Leistungsfähigkeit von Nutzern/Nutz...,NaN,NaN,released,2016-12-20T19:25:53Z,NaN,NaN,http://data.europa.eu/esco/concept-scheme/memb...,"Wiederherstellen der kognitiven, sensomotorisc..."
13887,KnowledgeSkillCompetence,http://data.europa.eu/esco/skill/fff0b074-5a76...,skill/competence,sector-specific,Beleuchtung in Transportgeräten einbauen,Beleuchtung in Transportmittel installieren,NaN,released,2016-12-20T20:03:21Z,NaN,NaN,http://data.europa.eu/esco/concept-scheme/skil...,Einbau von Beleuchtungselementen in Transportg...
13888,KnowledgeSkillCompetence,http://data.europa.eu/esco/skill/fff0e2cd-d0bd...,knowledge,sector-specific,Verarbeitung natürlicher Sprache,Verarbeitung natürlicher Sprachen\nNatural Lan...,NaN,released,2016-08-04T15:19:37Z,NaN,NaN,http://data.europa.eu/esco/concept-scheme/skil...,"Technologien, die es IKT-Geräten ermöglichen, ..."
13889,KnowledgeSkillCompetence,http://data.europa.eu/esco/skill/fff5bc45-b506...,skill/competence,cross-sector,Bauarbeiten koordinieren,NaN,NaN,released,2016-12-20T18:22:35Z,NaN,NaN,http://data.europa.eu/esco/concept-scheme/skil...,Koordinierung der Tätigkeiten mehrerer Bauarbe...


In [ ]:
skill_labels = skills['preferredLabel'].copy(deep=True)
termStore = {}
URI = 0
for i,sequence in enumerate(skill_labels):
    sequence = sp(sequence)
    label = ''
    for term in sequence:
        if not term.is_stop:
            term = term.lemma_
            term = term.lower()
            if term != '--':
                if term[0] == '-': term = term[1:]
                if term == '\xa0': continue
                label += term + ' '
                if termStore.get(term) == None: 
                    termStore[term] = URI
                    URI += 1
        skill_labels[i] = label[:-1]

In [ ]:
skill_labels

In [ ]:
sequenceStore = {}
for i,sequence in enumerate(skill_labels):
    URI = []
    sequence = sp(sequence)
    for term in sequence:
        URI.append(termStore[str(term)])
    sequenceStore[tuple(URI)] = (i,str(sequence))

In [ ]:
course_descriptions = pd.read_xml('./data/course-description/example.xml')['CS_DESC_LONG']
text = sp(course_descriptions[1])

sequence = []
results = []
for term in text:
    term = term.lemma_.lower()
    if term in termStore:
        sequence.append(termStore[term])
    else:
        if sequence != []:
            URI = []
            for i in sequence:
                URI.append(i)
                if tuple(URI) in sequenceStore: 
                    result = sequenceStore[tuple(URI)]
                else: break
            if result: results.append(result)
            result = None
        sequence = []
results

In [ ]:
for result in results:
    print(skills.loc[result[0]])

In [ ]:
# skill_labels[skill_labels.str.contains('gespräch')]
# skills[skills['description'].str.contains('Gespräche')]['description'].values

In [ ]:
# code from paper discard
# def annotate (text):
#     tt = sp(text)
#     s = p = 0
#     at = ''
#     a = []
#     for token in tt:
#         p += 1
#         if s > 1:
#             s -= 1
#             continue
#         elif token.lemma_ not in termStore:
#             at = at + ' ' + token
#         else: 
#             phrase,l = lookahead(tt[p+1:], [token], 1)
#             if phrase != []:
#                 URI = sequenceStore.get(phrase)
#                 if URI != None:
#                     a = a.append((p, phrase, URI))
#                     at += ' ' + phrase
#                     continue
#             at += ' ' + token
#         return a, at

# def lookahead(tt, fp, n):
#     if len(tt) == 0: return fp, n 
#     termFound = tt[0].lemma_.lower() in termStore
#     phraseFound = tuple(fp) in sequenceStore
#     if termFound or phraseFound:
#         fp.append(tt[0])
#         ph, l = lookahead(tt[1:], fp, n+1)
#         print(fp, ph)
#         if tuple(ph) in sequenceStore: return ph, l
#         elif phraseFound: return fp, n
#     return ([],n)